In [1]:
#Dependencies
import pandas as pd
import keras_tuner as kt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping

### Getting Data for feeding the model

In [ ]:
#Read in cleaned csv into a pandas dataframe
filename = "NONEXISTANT FILE.CSV"
try:
    df = pd.read_csv(f"../Data/{filename}")
except FileNotFoundError:
    raise FileNotFoundError(f"The file '{filename}' does not exist. Please make sure it is in the 'Data' folder.")

In [ ]:
#Split data for testing and training
yColName = "NONE"
X = df.drop(yColName, axis=1).values
y = df[yColName].values

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [ ]:
#Scale the data

#Create a StandardScaler instances
scaler = StandardScaler()

#Fit the StandardScaler
X_scaler = scaler.fit(X_train)

#Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Model Creation

In [ ]:
#Create model function for keras tuner
dimensions = 44
binaryOutput = True
MAX_NEURONS = 100
MAX_LAYERS = 10

def create_model(hp):
    nnModel = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nnModel.add(tf.keras.layers.Dense(units=hp.Int('first_units', min_value=1, max_value=MAX_NEURONS, step=2), activation=activation, input_dim=dimensions))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, MAX_LAYERS)):
        nnModel.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i), min_value=1, max_value=MAX_NEURONS, step=2), activation=activation))

    # Last layer for binary classification
    if(binaryOutput):  
        nnModel.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    elif (not binaryOutput):
        nnModel.add(tf.keras.layers.Dense(units=1, activation="linear"))

    # Compile the model
    nnModel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return nnModel

In [ ]:
#Use the tuner to generate the best model
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2
)

tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))
nn = tuner.get_best_models(num_models=1)[0]

In [ ]:
#Create an EarlyStopping callback
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

fit_model = nn.fit(X_train_scaled, y_train, epochs=500, callbacks=[early_stopping])

In [ ]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

#### All cells below are previous outputs of the cell above as we go through the process of optimization, along with what we changed